In [34]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [35]:
# prepare the dataset
bc = datasets.load_breast_cancer()  # binary classification dataset
X, Y = bc.data, bc.target
n_samples, n_features = X.shape
print(n_samples, n_features)

569 30


In [36]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1234)
X_train[0:2]

array([[1.288e+01, 1.822e+01, 8.445e+01, 4.931e+02, 1.218e-01, 1.661e-01,
        4.825e-02, 5.303e-02, 1.709e-01, 7.253e-02, 4.426e-01, 1.169e+00,
        3.176e+00, 3.437e+01, 5.273e-03, 2.329e-02, 1.405e-02, 1.244e-02,
        1.816e-02, 3.299e-03, 1.505e+01, 2.437e+01, 9.931e+01, 6.747e+02,
        1.456e-01, 2.961e-01, 1.246e-01, 1.096e-01, 2.582e-01, 8.893e-02],
       [1.113e+01, 2.244e+01, 7.149e+01, 3.784e+02, 9.566e-02, 8.194e-02,
        4.824e-02, 2.257e-02, 2.030e-01, 6.552e-02, 2.800e-01, 1.467e+00,
        1.994e+00, 1.785e+01, 3.495e-03, 3.051e-02, 3.445e-02, 1.024e-02,
        2.912e-02, 4.723e-03, 1.202e+01, 2.826e+01, 7.780e+01, 4.366e+02,
        1.087e-01, 1.782e-01, 1.564e-01, 6.413e-02, 3.169e-01, 8.032e-02]])

In [37]:
# scale the data
sc = StandardScaler()

# difference between fit transform and transform
# fit computes the mean and variance
# transform applies them to scale the data
# since we want both train and test to have same mean and variance
# we do not call fit again on the test data
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
X_train[0:2]

array([[-0.36180827, -0.26521011, -0.31715702, -0.46713841,  1.80382609,
         1.18174184, -0.51689239,  0.10653677, -0.39005152,  1.39140136,
         0.14370971, -0.12075458,  0.16013008, -0.13255126, -0.58631633,
        -0.12476339, -0.57865272,  0.10906824, -0.28187354, -0.1889237 ,
        -0.25710149, -0.24033176, -0.2441722 , -0.36688232,  0.5448709 ,
         0.24812607, -0.71088618, -0.07967528, -0.52798733,  0.2506337 ],
       [-0.8632675 ,  0.71560604, -0.85646012, -0.79668041, -0.05863438,
        -0.42846033, -0.51701741, -0.68142992,  0.7947534 ,  0.38824034,
        -0.45446828,  0.40088484, -0.43568602, -0.52155109, -1.16305609,
         0.27242031,  0.06745547, -0.23915919,  1.11303542,  0.35017293,
        -0.88943217,  0.38469608, -0.8880358 , -0.78968152, -1.04286301,
        -0.48243034, -0.5630865 , -0.76980239,  0.44312729, -0.20987332]])

In [38]:
# transform data into tensors
X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
Y_train = torch.from_numpy(Y_train.astype(np.float32))
Y_test = torch.from_numpy(Y_test.astype(np.float32))
X_train[0:2]

tensor([[-0.3618, -0.2652, -0.3172, -0.4671,  1.8038,  1.1817, -0.5169,  0.1065,
         -0.3901,  1.3914,  0.1437, -0.1208,  0.1601, -0.1326, -0.5863, -0.1248,
         -0.5787,  0.1091, -0.2819, -0.1889, -0.2571, -0.2403, -0.2442, -0.3669,
          0.5449,  0.2481, -0.7109, -0.0797, -0.5280,  0.2506],
        [-0.8633,  0.7156, -0.8565, -0.7967, -0.0586, -0.4285, -0.5170, -0.6814,
          0.7948,  0.3882, -0.4545,  0.4009, -0.4357, -0.5216, -1.1631,  0.2724,
          0.0675, -0.2392,  1.1130,  0.3502, -0.8894,  0.3847, -0.8880, -0.7897,
         -1.0429, -0.4824, -0.5631, -0.7698,  0.4431, -0.2099]])

In [39]:
# transform y into column vector
print(Y_train.shape)
Y_train = Y_train.view(Y_train.shape[0], 1)
Y_test = Y_test.view(Y_test.shape[0], 1)
print(Y_train.shape)

torch.Size([455])
torch.Size([455, 1])


In [40]:
# create model
class LogisticRegression(nn.Module):
    def __init__(self, input_size, output_size):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_size, output_size)
        
    def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred

In [41]:
model = LogisticRegression(n_features, 1)

In [42]:
# create loss and optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [43]:
# training loop
n_epochs = 1000
for epoch in range(n_epochs):
    # forward pass
    y_pred = model(X_train)
    # compute loss
    loss = criterion(y_pred, Y_train)
    # backward pass
    loss.backward()
    # update weights
    optimizer.step()
    # zero gradients
    optimizer.zero_grad()
    
    if epoch % 100 == 0:
        print(f'epoch {epoch + 1}: loss = {loss.item():.4f}')

epoch 1: loss = 0.6080
epoch 101: loss = 0.2363
epoch 201: loss = 0.1718
epoch 301: loss = 0.1426
epoch 401: loss = 0.1256
epoch 501: loss = 0.1142
epoch 601: loss = 0.1060
epoch 701: loss = 0.0996
epoch 801: loss = 0.0946
epoch 901: loss = 0.0905


In [44]:
# test
with torch.no_grad():
    y_pred = model(X_test)
    y_pred_class = y_pred.round()
    acc = y_pred_class.eq(Y_test).sum() / float(Y_test.shape[0])
    print(f'accuracy = {acc:.4f}')

accuracy = 0.9474
